In [ ]:
import pandas as pd
import plotly.graph_objects as go
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import matplotlib.pyplot as plt
import numpy as np

# Read the SpaceX launch data into a pandas DataFrame
df = pd.read_csv('spacex_launch.csv')

# Create a Dash application
app = dash.Dash(__name__)

# Get the unique launch sites from the DataFrame
launch_sites = df['Launch Site'].unique()

# Get the min and max payload values
min_payload = df['Payload Mass (kg)'].min()
max_payload = df['Payload Mass (kg)'].max()

# Create the options for the dropdown menu
dropdown_options = [{'label': 'All', 'value': 'All'}] + [{'label': site, 'value': site} for site in launch_sites]

# Set the application layout
app.layout = html.Div(
    children=[
        html.H1('SpaceX Launch Records Dashboard', style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
        html.Div(
            [
                "Select Launch Site: ",
                dcc.Dropdown(
                    id='site-dropdown',
                    options=dropdown_options,
                    value='All',
                    style={'height': '50px', 'font-size': 35}
                ),
            ],
            style={'font-size': 40}
        ),
        html.Br(),
        html.Div(
            [
                "Select Payload Range (kg): ",
                dcc.RangeSlider(
                    id='payload-slider',
                    min=min_payload,
                    max=max_payload,
                    step=1000,
                    value=[min_payload, max_payload],
                    marks={str(min_payload): str(min_payload), str(max_payload): str(max_payload)},
                    allowCross=False
                ),
            ],
            style={'font-size': 40}
        ),
        html.Br(),
        html.Div(dcc.Graph(id='success-pie-chart')),
        html.Div(dcc.Graph(id='success-payload-scatter-chart')),
    ]
)


# Callback function to update the success pie chart based on the selected launch site and payload range
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)

def update_success_pie_chart(selected_site, payload_range):
    min_payload = payload_range[0]
    max_payload = payload_range[1]

    if selected_site == 'All':
        # Filter the data based on the payload range
        filtered_data = df[(df['Payload Mass (kg)'] >= min_payload) & (df['Payload Mass (kg)'] <= max_payload)]
        class_counts = filtered_data[filtered_data['class'] == 1]['Launch Site'].value_counts()
    else:
        # Filter the data based on the selected launch site and payload range
        filtered_data = df[(df['Launch Site'] == selected_site) & (df['Payload Mass (kg)'] >= min_payload) & (df['Payload Mass (kg)'] <= max_payload)]
        class_counts = filtered_data['class'].value_counts()

    # Create the pie chart using Plotly
    fig = go.Figure(data=go.Pie(labels=class_counts.index, values=class_counts.values, hole=0.3))
    fig.update_layout(title='Class Proportion')
    
    return fig


# Callback function to update the success-payload-scatter-chart scatter plot based on the selected launch site and payload range
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')
)

def update_success_payload_scatter_chart(selected_site, payload_range):
    min_payload = payload_range[0]
    max_payload = payload_range[1]

    if selected_site == 'All':
        # Filter the data based on the payload range
        filtered_data = df[(df['Payload Mass (kg)'] >= min_payload) & (df['Payload Mass (kg)'] <= max_payload)]
        booster_versions = filtered_data['Booster Version'].unique()
    else:
        # Filter the data based on the selected launch site and payload range
        filtered_data = df[(df['Launch Site'] == selected_site) & (df['Payload Mass (kg)'] >= min_payload) & (df['Payload Mass (kg)'] <= max_payload)]
        booster_versions = filtered_data['Booster Version'].unique()

    # Create a color map for each booster version
    colors = plt.cm.viridis(np.linspace(0, 1, len(booster_versions)))

    # Create the scatter plot using Plotly
    fig = go.Figure()
    for i, version in enumerate(booster_versions):
        version_data = filtered_data[filtered_data['Booster Version'] == version]
        fig.add_trace(go.Scatter(
            x=version_data['Payload Mass (kg)'],
            y=version_data['class'],
            mode='markers',
            marker=dict(
                size=10,
                color=colors[i],  # Assign a color from the color map to each version
                showscale=False
            ),
            name=version,
            hovertext=version_data['Booster Version']
        ))

    fig.update_layout(
        title='Payload vs. Class',
        xaxis_title='Payload Mass (kg)',
        yaxis_title='Class'
    )

    return fig

# Run the application
if __name__ == '__main__':
    app.run_server()


<ipython-input-1-070982bdf2bb>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-070982bdf2bb>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jun/2023 16:43:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 16:43:43] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 16:43:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 16:43:43] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 16:43:43] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 16:43:43] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 16:43:43] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 16:43:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [12/Jun/2023 16:43:44] "POST /_dash-update-component HTTP/1.1" 200 -
